## Importing Librarys and Setting up the Environment 



In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os 
import pandas as pd 
import numpy as np
import copy
import re
import string
input_path = 'gdrive/MyDrive/2020_21_2/Temalabor_1/processed_popdata/'
files = os.listdir(input_path)

## Function definitions
1. Function defintion of trim_all_columns(df) 


In [ ]:
# Function1.: Trimming  all leading and trailing whitespaces, apply str to the whole dfs

def trim_all_columns(df):

    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    
    df = df.applymap(trim_strings)
    df = df.applymap(str) #converting the whole df datatype into string
    return df


## Premliminary steps 

1.   Loading CSVS
2.   Trimming all leading and trailing whitespaces using 

```
trim_all_columns(df) 
```






In [ ]:
# 1. Loading CSVS

full_pop = pd.read_csv(input_path + 'populationData_HU_1985-2020_full.csv', dtype=str)
basic_pop = pd.read_csv(input_path + 'populationData_HU_1985-2020_basic.csv', dtype=str)
election=pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/election.csv', dtype=str)


In [ ]:
# 2. Trimming all leading and trailing whitespaces using trim_all_columns(df) aka Using Function1.:

full_pop = trim_all_columns(full_pop)
basic_pop = trim_all_columns(basic_pop)
election = trim_all_columns(election)

## EDA 


### Setting up for election

In [ ]:
elecYear_pop = full_pop[ (full_pop['year'] == '1990') | (full_pop['year'] == '1994') 
                               | (full_pop['year'] == '1998') | (full_pop['year'] == '2002') 
                               | (full_pop['year'] == '2006') | (full_pop['year'] == '2010') 
                               | (full_pop['year'] == '2014') | (full_pop['year'] == '2018') ]

In [ ]:
elec_years = sorted(elecYear_pop.year.unique())
print('year' + ' ----- ' + 'num of entries' + ' ----- ' + 'unique num of geonameid' + 'unique num of geoid' )
for year in elec_years: 
  print( year, '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]),  
        '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]['geonameid'].unique()), 
        '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]['geoid'].unique()) ) 

year ----- num of entries ----- unique num of geonameidunique num of geoid
1990 ----- 3091 ----- 3091 ----- 3091
1994 ----- 3149 ----- 3149 ----- 3149
1998 ----- 3153 ----- 3153 ----- 3153
2002 ----- 3167 ----- 3167 ----- 3167
2006 ----- 3174 ----- 3174 ----- 3174
2010 ----- 3176 ----- 3176 ----- 3176
2014 ----- 3177 ----- 3177 ----- 3177
2018 ----- 3177 ----- 3177 ----- 3177


In [ ]:
elecYear_pop

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
5,1990,Aba,4103,1650,9692,1512,47.0343556977497,18.524754962635
9,1994,Aba,4202,1650,9692,1512,47.0343556977497,18.524754962635
13,1998,Aba,4332,1650,9692,1512,47.0343556977497,18.524754962635
17,2002,Aba,4526,1650,9692,1512,47.0343556977497,18.524754962635
21,2006,Aba,4654,1650,9692,1512,47.0343556977497,18.524754962635
...,...,...,...,...,...,...,...,...
113340,2002,Ősi,2244,1650,9759,2160,47.1539446896795,18.190410278261
113344,2006,Ősi,2190,1650,9759,2160,47.1539446896795,18.190410278261
113348,2010,Ősi,2169,1650,9759,2160,47.1539446896795,18.190410278261
113352,2014,Ősi,2089,1650,9759,2160,47.1539446896795,18.190410278261


### **Examining election dfs**



1.   Creating a variable named *year* from stripping *eventdate* for the purpose of merging with **population** on *year* (and *settlement*)
2.   List item



In [ ]:
#1. Creating a variable named year from stripping eventdate 

election=pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/election.csv', dtype=str)
election = trim_all_columns(election)

year = election['eventdate'].str[0:4] 
election['year'] = year 
cols = election.columns.tolist() # listing all the columns, year is the last column at this moment 
cols = cols[-1:] + cols[:-1]  #putting year as a first col 
election = election[cols] #save this into the election df 

In [ ]:
election

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047408,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0
1047409,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0
1047410,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0
1047411,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0


In [ ]:
# Searching for '16' string in partyids => Pay attention to '166' which is the partyid of Magyar Fuggetlensegi Part 
lst = list()
for i in election.partyids.unique(): 
  if '16' in i:
    lst.append(i)

print(lst)

['16', '166', '269 | 16', '16 | 269', '16 | 26', '26 | 16']


In [ ]:
#Trial: If we had used df.str.match(...), it woud only show us 5 instances instead of 8 in the following case
settlementid996 = election[ (election.settlementid == '996') & ( (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20835,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,54,Fiatal Demokraták Szövetsége,16,26
148260,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,41,FIDESZ,16,30
282930,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,25,Fidesz Magyar Polgári Párt,16,30
398760,2002,12,2002-04-16,"2002, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9225,85,MDF | FIDESZ,269 | 16,110
485433,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,170
560795,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,83,KDNP | FIDESZ,26 | 16,172
655851,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,83,FIDESZ | KDNP,16 | 26,188
855990,2018,48,2018-04-08,"2018, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,69185,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,292


In [ ]:
election.eventdate.unique()  #There are two events in 1998. Need to drop instances in 1998-05-10 which are also in 1998-05-24 

array(['1990-03-25', '1994-05-08', '1998-05-10', '1998-05-24',
       '2002-04-16', '2006-04-09', '2010-04-11', '2014-04-06',
       '2018-04-08'], dtype=object)

In [ ]:
elec98_1stR = election[election['eventdate'] == '1998-05-10'] # Filter only those when eventdate == '1998-05-10'
elec98_2ndR = election[election['eventdate'] == '1998-05-24'] # Filter only those when eventdate == '1998-05-24'. In 98, there were some wards in settlements where 2nd round election had to be held. We are only interested in second round of those. 

wardnameid_lst_elec98_2ndR = list(elec98_2ndR.wardnameid.unique()) # listing the unique wardnameid we come across in the second round of 98's election
wardnameid_dict_elec98_2ndR = {'wardnameid' : wardnameid_lst_elec98_2ndR} #putting them into a dict with 'wardenameid' key. It's essenital when we wanna find those wardnameid from the 2nd round of 98's election in the first round. (We wanna get rid of those in round 1 of 98's election)

In [ ]:
len(wardnameid_lst_elec98_2ndR) #there are 1077 unique wardnameid in the 2nd round of 98s election 

1077

In [ ]:
row_mask = elec98_1stR.isin(wardnameid_dict_elec98_2ndR).any(1)  #PD Series of booelean value where True pertains to values rows where the value of wardnameid_dict_elec98_2ndR can also be found in wardnameid_dict_elec98_2ndR
print( len(elec98_1stR[row_mask].wardnameid.unique()) )  #checking whether we have 1077 unique wardnameid in elec98_1stR[row_mask]. Yes, indeed. 

print( sorted(elec98_1stR[row_mask].wardnameid.unique()) == sorted(elec98_2ndR.wardnameid.unique()) ) #Checking whether the content of elec98_1stR[row_mask].wardnameid.unique() and elec98_2ndR.wardnameid.unique() is the same => yes, indeed.

elec98_1stR_indices_to_delete = elec98_1stR[row_mask].index # we wanna get the indices of elec98_1stR[row_mask] in order to axe them out in the original election df. 
print( len(elec98_1stR_indices_to_delete) ) #There are 12563 rows which contains in both the wardnameid from elec98_1stR and wardnameid from elec98_2ndR

1077
True
12563


In [ ]:
election = election.drop(elec98_1stR_indices_to_delete) #Dropping those indices from election df and save it again in election  
election.reset_index(drop = True, inplace = True) # reset the indexing of the new election 

In [ ]:
election

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034845,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0
1034846,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0
1034847,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0
1034848,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0


In [ ]:
# NEW

electionPopulation =  pd.merge(election, elecYear_pop, left_on=['settlementid', 'year'], right_on=['geoid', 'year'])
votePop_elig = electionPopulation['value'].astype(np.int64) * 0.8
votePop_elig = votePop_elig.round().astype(np.int64)
electionPopulation['votePop_elig'] = votePop_elig
electionPopulation = electionPopulation.applymap(str)
electionPopulation

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034845,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034846,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034847,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034848,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56


In [ ]:
len(electionPopulation.name.unique())

3188

### Splitting up the merged electionPopulation df

Fidesz's *partyid==16* so let's dissect our **election** df into **FIDESZ**  **NotFidesz** dfs



```
(election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) 
```



In [ ]:
eFidesz = election[ (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) ]
eNotFidesz = election[~( (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) ) ]

In [ ]:
print(len(eFidesz)) #number of rows in eFidesz df 
print(len(eNotFidesz)) ##number of rows in eNotFidesz df
print(len(eFidesz) + len(eNotFidesz) == len(election))  #Check whether the sum of splitted dfs equal to the original 

86033
948817
True


As we can see there are significant data losses in **electionpopulation** compared to **election**. 
**electionpoulation** includes poulation data and other data as well. 

In [ ]:
Fidesz = electionPopulation[ (electionPopulation.partyids.str.contains('^16$')) | (electionPopulation.partyids.str.contains('^16 \|')) | (electionPopulation.partyids.str.contains('\| 16$')) ]
Fidesz.reset_index(drop=True, inplace=True)

NotFidesz = electionPopulation[~( (electionPopulation.partyids.str.contains('^16$')) | (electionPopulation.partyids.str.contains('^16 \|')) | (electionPopulation.partyids.str.contains('\| 16$')) ) ]
NotFidesz.reset_index(drop=True, inplace=True)

print(len(Fidesz)) #number of rows in Fidesz df 
print(len(NotFidesz)) ##number of rows in NotFidesz df
print(len(Fidesz) + len(NotFidesz) == len(electionPopulation))  #Check whether the sum of splitted dfs equal to the original 

86033
948817
True


In [ ]:
Fidesz

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,54,Fiatal Demokraták Szövetsége,16,23,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,54,Fiatal Demokraták Szövetsége,16,16,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,54,Fiatal Demokraták Szövetsége,16,27,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,54,Fiatal Demokraták Szövetsége,16,34,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,54,Fiatal Demokraták Szövetsége,16,30,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86028,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,37,Zalasárszeg,127,1650,15763,97,46.4946504867345,17.0821631084216,102
86029,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,223,Zalaszentbalázs,798,1650,15762,1628,46.5909416707031,16.9189750148023,638
86030,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,98,Zalaszentjakab,320,1650,17105,1581,46.486657871767,17.1297592011168,256
86031,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,37,Zalaújlak,104,1650,17104,723,46.5607974894449,17.0798804702104,83


In [ ]:
NotFidesz

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948812,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
948813,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
948814,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
948815,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56


In [ ]:
#Checking the unique partyids => it show us what we want to achieve 
Fidesz.partyids.unique()

array(['16', '269 | 16', '16 | 269', '16 | 26', '26 | 16'], dtype=object)

In [ ]:
#Checking whether any partidys pertaining to Fidesz leaked to NotFidesz dataframe => For god sake, no. Only 166, but this is the party id of Magyar Fuggetlensegi Part. So far, so good. 
lst2 = list()
for i in NotFidesz.partyids.unique(): 
  if '16' in i:
    lst2.append(i)

print(lst2)

['166']


In [ ]:
#Checking the # of unique geoids 
len(NotFidesz.geoid.unique()) == len(Fidesz.geoid.unique())

True

In [ ]:
#Comparing the # of unique settlementid in both Fidesz and NotFidesz by election years 
print('1990 -----', len(Fidesz[Fidesz.year == '1990'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '1990'].settlementid.unique()))
print('1994 -----', len(Fidesz[Fidesz.year == '1994'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '1994'].settlementid.unique()))
print('1998 -----', len(Fidesz[Fidesz.year == '1998'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '1998'].settlementid.unique()))
print('2002 -----', len(Fidesz[Fidesz.year == '2002'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2002'].settlementid.unique()))
print('2006 -----', len(Fidesz[Fidesz.year == '2006'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2006'].settlementid.unique()))
print('2010 -----', len(Fidesz[Fidesz.year == '2010'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2010'].settlementid.unique()))
print('2014 -----', len(Fidesz[Fidesz.year == '2014'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2014'].settlementid.unique()))
print('2018 -----', len(Fidesz[Fidesz.year == '2018'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2018'].settlementid.unique()))

1990 ----- 3089 3089
1994 ----- 3135 3135
1998 ----- 3153 3153
2002 ----- 3157 3157
2006 ----- 3167 3167
2010 ----- 3174 3174
2014 ----- 3176 3176
2018 ----- 3177 3177


In [ ]:
#Comparing the CONTENTS of uniduqe settlementids in both Fidesz and NotFidesz => turns out all of them have the same settlementid contents in every election years

print('1990 -----', sorted(Fidesz[Fidesz.year == '1990'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '1990'].settlementid.unique()) )
print('1994 -----', sorted(Fidesz[Fidesz.year == '1994'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '1994'].settlementid.unique()) )
print('1998 -----', sorted(Fidesz[Fidesz.year == '1998'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '1998'].settlementid.unique()) )
print('2002 -----', sorted(Fidesz[Fidesz.year == '2002'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2002'].settlementid.unique()) )
print('2006 -----', sorted(Fidesz[Fidesz.year == '2006'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2006'].settlementid.unique()) )
print('2010 -----', sorted(Fidesz[Fidesz.year == '2010'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2010'].settlementid.unique()) )
print('2014 -----', sorted(Fidesz[Fidesz.year == '2014'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2014'].settlementid.unique()) )
print('2018 -----', sorted(Fidesz[Fidesz.year == '2018'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2018'].settlementid.unique()) )

1990 ----- True
1994 ----- True
1998 ----- True
2002 ----- True
2006 ----- True
2010 ----- True
2014 ----- True
2018 ----- True


### ```settlementid```-ra vonatkozo **nepessegszamot** es **szavazasra jogosultak szamat** es az aktualis szavazat szamot is vedd bele, ami a ```value```, ```votePop_elig``` es  ```vote``` oszlop alatt van letarolva azt belekene venni az alabbi adattablakba a szemleletesseg kedveert. 

### Illetve a kulonbozo szavazati aranyok szamolasan kerlek hozz letre valami uj oszlopnevet az aranyossagra, mert a ```vote``` tartsuk meg az aktualis szavazatszamra

### Some basic calculations related to votes 

In [ ]:
NotFidesz['vote']=NotFidesz['vote'].astype(np.int64)
votes_NotFidesz=NotFidesz[['year','settlementid','vote']].groupby(['year','settlementid']).sum()
votes_NotFidesz=votes_NotFidesz.reset_index()
votes_NotFidesz

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,settlementid,vote
0,1990,10,827
1,1990,100,44078
2,1990,1000,1314
3,1990,1001,113
4,1990,1002,1037
...,...,...,...
25223,2018,995,581
25224,2018,996,124
25225,2018,997,17
25226,2018,998,264


In [ ]:
Fidesz['vote']=Fidesz['vote'].astype(np.int64)
votes_Fidesz=Fidesz[['year','settlementid','vote']].groupby(['year','settlementid']).sum()
votes_Fidesz=votes_Fidesz.reset_index()
votes_Fidesz

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,settlementid,vote
0,1990,10,102
1,1990,100,3563
2,1990,1000,176
3,1990,1001,6
4,1990,1002,79
...,...,...,...
25223,2018,995,1221
25224,2018,996,292
25225,2018,997,21
25226,2018,998,188


In [ ]:
votes_NotFidesz['vote'] = votes_NotFidesz['vote'].astype(np.float64)
votes_Fidesz['vote'] = votes_Fidesz['vote'].astype(np.float64)
# votes_rating=votes_NotFidesz.copy()   #Shallow copy is not really desirable, in this case this would not cause a problem. 
votes_NotFidesz_per_Fidesz = copy.deepcopy(votes_NotFidesz)
votes_NotFidesz_per_Fidesz['vote']=votes_NotFidesz['vote']/votes_Fidesz['vote']
votes_NotFidesz_per_Fidesz

,year,settlementid,vote
0,1990,10,8.107843
1,1990,100,12.371036
2,1990,1000,7.465909
3,1990,1001,18.833333
4,1990,1002,13.126582
...,...,...,...
25223,2018,995,0.475839
25224,2018,996,0.424658
25225,2018,997,0.809524
25226,2018,998,1.404255


In [ ]:
votes_rating_Fidesz = copy.deepcopy(votes_Fidesz)
votes_rating_Fidesz['vote'] = votes_Fidesz['vote'] / (votes_NotFidesz['vote']+votes_Fidesz['vote'])
votes_rating_Fidesz

,year,settlementid,vote
0,1990,10,0.109795
1,1990,100,0.074789
2,1990,1000,0.118121
3,1990,1001,0.050420
4,1990,1002,0.070789
...,...,...,...
25223,2018,995,0.677580
25224,2018,996,0.701923
25225,2018,997,0.552632
25226,2018,998,0.415929


In [ ]:
votes_rating_NotFidesz = copy.deepcopy(votes_NotFidesz)
votes_rating_NotFidesz['vote'] = votes_NotFidesz['vote']/(votes_NotFidesz['vote']+votes_Fidesz['vote'])
votes_rating_NotFidesz

,year,settlementid,vote
0,1990,10,0.890205
1,1990,100,0.925211
2,1990,1000,0.881879
3,1990,1001,0.949580
4,1990,1002,0.929211
...,...,...,...
25223,2018,995,0.322420
25224,2018,996,0.298077
25225,2018,997,0.447368
25226,2018,998,0.584071


In [ ]:
votes_Fidesz['vote'] = votes_Fidesz['vote'].astype(str)
votes_NotFidesz['vote'] = votes_NotFidesz['vote'].astype(str)

votes_NotFidesz_per_Fidesz['vote'] = votes_NotFidesz_per_Fidesz['vote'].astype(str)

votes_rating_NotFidesz['vote'] = votes_rating_NotFidesz['vote'].astype(str)
votes_rating_Fidesz['vote'] = votes_rating_Fidesz['vote'].astype(str)

In [ ]:
votes_rating_Fidesz_1990=votes_rating_Fidesz[votes_rating_Fidesz['year']=='1990']
votes_rating_Fidesz_1994=votes_rating_Fidesz[votes_rating_Fidesz['year']=='1994']
votes_rating_Fidesz_1998=votes_rating_Fidesz[votes_rating_Fidesz['year']=='1998']
votes_rating_Fidesz_2002=votes_rating_Fidesz[votes_rating_Fidesz['year']=='2002']
votes_rating_Fidesz_2006=votes_rating_Fidesz[votes_rating_Fidesz['year']=='2006']
votes_rating_Fidesz_2010=votes_rating_Fidesz[votes_rating_Fidesz['year']=='2010']
votes_rating_Fidesz_2014=votes_rating_Fidesz[votes_rating_Fidesz['year']=='2014']
votes_rating_Fidesz_2018=votes_rating_Fidesz[votes_rating_Fidesz['year']=='2018']

votes_rating_NotFidesz_1990=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='1990']
votes_rating_NotFidesz_1994=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='1994']
votes_rating_NotFidesz_1998=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='1998']
votes_rating_NotFidesz_2002=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='2002']
votes_rating_NotFidesz_2006=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='2006']
votes_rating_NotFidesz_2010=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='2010']
votes_rating_NotFidesz_2014=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='2014']
votes_rating_NotFidesz_2018=votes_rating_NotFidesz[votes_rating_NotFidesz['year']=='2018']

# Ezek lennenek az alabbi feladatiad
Ellenorizd le kerlek, h:

*   Adott ```settlementid```-ra a ``` votes_rating_Fidesz['vote'] ```es ```votes_rating_NotFidesz['vote']``` osszege mindenhol 1-e, es ha van olyan sor ahol nem, akkor nyomd ki (ez tenyleg csak sanity check, h veltelenul nem tortent-e galiba). Es ahogy fent emlitettem a ```vote``` oszlop alatt betarolt valtozoknak adj egy masik valtozonevet 

*   Adott ```settlementid```-ra ```(votes_NotFidesz['vote']+votes_Fidesz['vote'])``` 
    1.   Atlepi-e az adott ```settlementid```-ra vonatkozo **nepessegszamot**, ami a ```value```  oszlop alatt van letarolva. Ha igen, akkor nyomd ki, h pontosan melyik settlement id-k lennenek  a hozzaja tartozo adataival (azaz df-et nyomj ki). Ha igen, akkor valahol nem pontos az adatunk vagy Szakadate, ezt kene kiszurni ezzel a lepessel. 

    2.   Atlepi-e az adott ```settlementid```-ra vonatkozo **szavazasra jogosultak szamat**, ami  ```votePop_elig```  oszlop alatt van letarolva es megallapodas szerint ugy kaptuk meg, hogy a nepessegszam 80%-a kb.  Ha igen, akkor nyomd ki, h pontosan melyik settlement id-k lennenek  a hozzaja tartozo adataival (azaz df-et nyomj ki). Ez par fontos dolog miatt fontos, hogy lassuk melyik telepulesnel lepi at a ```(votes_NotFidesz['vote']+votes_Fidesz['vote'])``` ezt a ```votePop_elig```. 

* A Budapestra vonatkozo adatokat aggregalni kellene hasonloan Fidesz es NotFideszes modszerre. A vegen nekunk az kell, h egy sort kapjunk u.h ott legyenek az aranyok ami kell nekunk es a hozzajatarozo nepessegszam es a szavazasra jogosultak szama. 






 